In [1]:
import pandas as pd
import numpy as np
import os
import csv
import math

In [3]:
from haversine import haversine, Unit
from haversine import haversine_vector, Unit

392.2172595594006

In [4]:
os.getcwd()
os.chdir("D:/WORK/Innovation/huff_gravity/")

In [5]:
file = pd.read_excel('Huff_Gravity_Data_30012020.xlsx')
store = pd.read_excel('Store_Data.xlsx')

In [6]:
store.head(5)

,Location,Lat,Lng,store size
0,A,32.64,-95.82,100000
1,B,32.50,-96.20,150000
2,C,32.93,-96.79,250000
3,D,33.11,-96.69,175000
4,E,31.13,-95.50,125000


In [40]:
s_size = list(store['store size'])

In [7]:
file.head(5)

,Zipcode,Total_Poplation,Total_Households,Total_Families,Latitude,Longitude
0,75001,12414,8287,2933,32.961290,-96.83751
1,75002,63140,22029,18423,33.092846,-96.62447
2,75006,46364,17780,11952,32.960374,-96.89163
3,75007,51624,19109,14658,33.005420,-96.89773
4,75009,8785,3532,2895,33.327927,-96.76129


In [16]:
dist = []
for i in range(len(file)):
    dist1 = []
    for j in range(len(store)):
        t1 = tuple(file.loc[i,['Latitude', 'Longitude']])
        t2 = tuple(store.loc[j,['Lat', 'Lng']])
        h = haversine(t1,t2)
        dist1.append(h)
    dist.append(dist1)

In [20]:
dist[1]

[90.44741520000082,
 76.93998863152328,
 23.79320479347019,
 6.395119648647436,
 242.59169147834166]

In [23]:
dist_df = pd.DataFrame(dist, columns=['store1', 'store2', 'store3', 'store4', 'store5'])

In [27]:
dist_df.head()

,store1,store2,store3,store4,store5
0,101.591114,78.657307,5.635586,21.506119,239.486995
1,90.447415,76.939989,23.793205,6.395120,242.591691
2,106.307527,82.498218,10.066964,25.101603,242.125040
3,108.593873,86.115383,13.089539,22.583499,246.639305
4,116.444781,105.926342,44.328216,25.123469,271.665695


In [72]:
df = file.merge(dist_df, on=file.index)

df = df.drop(columns=['key_0'], axis=1)

df.head()

,Zipcode,Total_Poplation,Total_Households,Total_Families,Latitude,Longitude,store1,store2,store3,store4,store5
0,75001,12414,8287,2933,32.961290,-96.83751,101.591114,78.657307,5.635586,21.506119,239.486995
1,75002,63140,22029,18423,33.092846,-96.62447,90.447415,76.939989,23.793205,6.395120,242.591691
2,75006,46364,17780,11952,32.960374,-96.89163,106.307527,82.498218,10.066964,25.101603,242.125040
3,75007,51624,19109,14658,33.005420,-96.89773,108.593873,86.115383,13.089539,22.583499,246.639305
4,75009,8785,3532,2895,33.327927,-96.76129,116.444781,105.926342,44.328216,25.123469,271.665695


In [42]:
df['store1'].head()

0    101.591114
1     90.447415
2    106.307527
3    108.593873
4    116.444781
Name: store1, dtype: float64

In [73]:
for i in range(len(store)):
    df[str('attraction_'+str(i+1))] = s_size[i]/df.iloc[:,i+6]**2

In [155]:
df.iloc[:,25:].head()

,lower_4,lower_5,upper_1,upper_2,upper_3,upper_4,upper_5,Weighted_Pop_1,Weighted_Pop_2,Weighted_Pop_3,...,Weighted_House_1,Weighted_family_1,Weighted_House_2,Weighted_family_2,Weighted_House_3,Weighted_family_3,Weighted_House_4,Weighted_family_4,Weighted_House_5,Weighted_family_5
0,0.0,0.0,0.1,0.1,1.0,0.1,0.1,620.70,620.70,11793.30,...,414.35,146.65,414.35,146.65,7872.65,2786.35,414.35,146.65,414.35,146.65
1,0.8,0.0,0.1,0.1,0.1,0.9,0.1,3157.00,3157.00,3157.00,...,1101.45,921.15,1101.45,921.15,1101.45,921.15,18724.65,15659.55,1101.45,921.15
2,0.0,0.0,0.1,0.1,0.9,0.1,0.1,2318.20,2318.20,39409.40,...,889.00,597.60,889.00,597.60,15113.00,10159.20,889.00,597.60,889.00,597.60
3,0.1,0.0,0.1,0.1,0.8,0.2,0.1,2581.20,2581.20,38718.00,...,955.45,732.90,955.45,732.90,14331.75,10993.50,2866.35,2198.70,955.45,732.90
4,0.6,0.0,0.1,0.1,0.3,0.7,0.1,439.25,439.25,2196.25,...,176.60,144.75,176.60,144.75,883.00,723.75,2295.80,1881.75,176.60,144.75


In [76]:
# addition of all attraction parameters
df['Tot_attraction'] = df.loc[:,['attraction_1','attraction_2','attraction_3','attraction_4','attraction_5']].sum(axis=1)

In [79]:
for i in range(len(store)):
    df[str('prob_'+str(i+1))] = df[str('attraction_'+str(i+1))]/df['Tot_attraction']

In [94]:
index = np.linspace(start=0, stop=0.9, num=10)
index

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [100]:
for i in range(len(store)):
    for j in range(len(index)):
            df.loc[(df[str('prob_'+str(i+1))] > index[j]) & (df[str('prob_'+str(i+1))] <= index[j]+0.1),str('lower_'+str(i+1))] = index[j]
            df.loc[(df[str('prob_'+str(i+1))] > index[j]) & (df[str('prob_'+str(i+1))] <= index[j]+0.1),str('upper_'+str(i+1))] = index[j]+0.1

In [152]:
a = []
b = []
c = []

for i in range(len(store)):
    df[str('Weighted_Pop_'+ str(i+1))] = df['Total_Poplation']*(df[str('upper_'+str(i+1))]+df[str('lower_'+str(i+1))])/2
    df[str('Weighted_House_'+ str(i+1))] = df['Total_Households']*(df[str('upper_'+str(i+1))]+df[str('lower_'+str(i+1))])/2
    df[str('Weighted_family_'+ str(i+1))] = df['Total_Families']*(df[str('upper_'+str(i+1))]+df[str('lower_'+str(i+1))])/2
    a.append(df[str('Weighted_Pop_'+ str(i+1))].sum()/df['Total_Poplation'].sum())
    b.append(df[str('Weighted_House_'+ str(i+1))].sum()/df['Total_Households'].sum())
    c.append(df[str('Weighted_family_'+ str(i+1))].sum()/df['Total_Families'].sum())

In [165]:
summary = pd.DataFrame([a,b,c]).T
summary.columns = ['perc_pop','perc_house','perc_family']

In [169]:
summary.index = ['store1','store2','store3','store4','store5']

In [171]:
summary*100

,perc_pop,perc_house,perc_family
store1,10.350639,10.211919,10.206879
store2,14.051925,13.901707,13.970002
store3,33.306030,33.645707,33.193064
store4,18.502757,18.593188,18.913333
store5,24.349058,24.331049,24.382564
